# Prepare mitosis time series data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path
# from livecell_tracker import segment
from livecell_tracker import core
from livecell_tracker.core import datasets
from livecell_tracker.core.datasets import LiveCellImageDataset, SingleImageDataset
from skimage import measure
from livecell_tracker.core import SingleCellTrajectory, SingleCellStatic

In [ ]:
sample_json_dir = Path("./EBSS_starvation_24h_xy16_annotation")
sample_dataset_dir = sample_json_dir / "datasets"
class_subfolders = ["mitosis", "apoptosis", "normal"]
# sample_paths = glob.glob(str(sample_json_dir / "*.json"))

class_samples = {}
for subfolder in class_subfolders:
    class_samples[subfolder] = []
    sample_paths = glob.glob(str(sample_json_dir / subfolder / "*.json"))
    for sample_path in sample_paths:
        sample = SingleCellStatic.load_single_cells_json(sample_path)
        class_samples[subfolder].append(sample)

In [ ]:
class_samples

Automatically prepare normal samples

require tracking done

In [ ]:
# get all scs from class_samples not in normal class
exclude_scs = []
total_non_normal_samples = 0
for class_name, samples in class_samples.items():
    if class_name != "normal":
        for sample in samples:
            exclude_scs.extend(sample)
            total_non_normal_samples += 1

exclude_scs = set(exclude_scs)

load all scs

In [ ]:
all_scs_json_path = "./datasets/test_scs_EBSS_starvation/tmp_corrected_scs.json"
all_scs = SingleCellStatic.load_single_cells_json(all_scs_json_path)

In [ ]:
import json
from livecell_tracker.core.single_cell import SingleCellTrajectoryCollection
from livecell_tracker.track.sort_tracker_utils import (
    track_SORT_bbox_from_scs
)
# with open("./EBSS_starvation_24h_xy16_annotation/single_cell_trajectory_collection.json", "r") as file:
#     json_dict = json.load(file)
# sctc = SingleCellTrajectoryCollection().load_from_json_dict(json_dict)
sctc = track_SORT_bbox_from_scs(all_scs, raw_imgs=all_scs[0].img_dataset, min_hits=3, max_age=3)

In [ ]:
objective_sample_num = total_non_normal_samples * 10

normal_frame_len_range = (3, 10)
counter = 0
normal_samples = []
max_trial_counter = 100000
while counter < objective_sample_num and max_trial_counter > 0:
    # randomly select a sct from sctc
    # generate a list of scs
    track_id = np.random.choice(list(sctc.track_id_to_trajectory.keys()))  
    sct = sctc.get_trajectory(track_id)
    # randomly select a length
    frame_len = np.random.randint(*normal_frame_len_range)
    # generate a sample
    times = list(sct.timeframe_to_single_cell.keys())
    times = sorted(times)
    if len(times) <= frame_len:
        continue
    start_idx = np.random.randint(0, len(times) - frame_len)
    start_time = times[start_idx]
    end_time = times[start_idx + frame_len - 1]

    sub_sct = sct.subsct(start_time, end_time)

    is_some_sc_in_exclude_scs = False
    for time, sc in sub_sct.timeframe_to_single_cell.items():
        if sc in exclude_scs:
            is_some_sc_in_exclude_scs = True
            break
    if is_some_sc_in_exclude_scs:
        continue
    
    new_sample = []
    for time, sc in sub_sct.timeframe_to_single_cell.items():
        new_sample.append(sc)
    normal_samples.append(new_sample)
    counter += 1
    max_trial_counter -= 1

normal_samples[:2]

In [ ]:
class_samples["normal"].extend(normal_samples)

## Prepare videos and annotations for MMDetection

In [ ]:
classes = class_samples.keys()
classes

In [ ]:
from livecell_tracker.core.utils import gray_img_to_rgb, rgb_img_to_gray
from livecell_tracker.preprocess.utils import normalize_img_to_uint8

In [ ]:
[len(sample) for sample in normal_samples]

In [ ]:
from livecell_tracker.track.classify_utils import video_frames_and_masks_from_sample, combine_video_frames_and_masks

In [ ]:
from typing import List
import cv2
import numpy as np
import pandas as pd

def gen_mp4_from_frames(video_frames, output_file, fps):
    # Define the output video file name and properties
    frame_size = video_frames[0].shape[:2][::-1]  # reverse the order of width and height

    # Create a VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_file), fourcc, fps, frame_size)
    # Write each frame to the output video
    for frame in video_frames:
        out.write(frame)
    out.release()

def gen_samples_mp4s(sc_samples: List[List[SingleCellStatic]], class_label, output_dir, fps = 1, padding_pixels=50):
    """
    Generate mp4 videos and masks from a list of SingleCellStatic samples.
    Args:
        sc_samples: A list of SingleCellStatic samples.
        class_label: A string representing the class label of the samples.
        output_dir: A Path object representing the directory to save the generated videos and masks.
        fps: An integer representing the frames per second of the generated videos.
        padding_pixels: An integer representing the number of pixels to pad around the cells in the generated videos and masks.
    Returns:
        A dictionary containing the file paths of the generated videos, masks, and combined videos.
    """
    res_paths = {
        "video": [],
        "mask": [],
        "combined": []
    }
    for i, sample in enumerate(sc_samples):
        output_file = output_dir / (f'{class_label}_{i}_padding-{padding_pixels}.mp4')
        mask_output_file = output_dir / (f'{class_label}_{i}_mask_padding-{padding_pixels}.mp4')
        combined_output_file = output_dir / (f'{class_label}_{i}_combined_padding-{padding_pixels}.mp4')
        
        # record video file path and class label
        video_frames, video_frame_masks = video_frames_and_masks_from_sample(sample, padding_pixels=padding_pixels)
        combined_frames = combine_video_frames_and_masks(video_frames, video_frame_masks)

        # for debug
        print("len video_frames: ", len(video_frames))
        print("len masks video: ", len(video_frame_masks))
        print("len combined_frames: ", len(combined_frames))

        gen_mp4_from_frames(video_frames, output_file, fps=fps)
        gen_mp4_from_frames(video_frame_masks, mask_output_file, fps=fps)
        gen_mp4_from_frames(combined_frames, combined_output_file, fps=fps)
        res_paths["video"].append(output_file)
        res_paths["mask"].append(mask_output_file)
        res_paths["combined"].append(combined_output_file)
    return res_paths


ver = 4
data_dir = Path(f'notebook_results/mmaction_train_data_v{ver}')
class_labels = ['mitosis', 'apoptosis', 'normal']
class_label = "mitosis"
frame_types = ["video", "mask", "combined"]
selected_frame_type = "video" 

padding_pixels = [0, 20, 40, 50, 100]
mmaction_df = pd.DataFrame(columns=["path", "label_index", "padding_pixels", "frame_type"])
for class_label in class_labels:
    output_dir = Path(data_dir) / "videos"
    output_dir.mkdir(exist_ok=True, parents=True)
    video_frames_samples = class_samples[class_label]
    for padding_pixel in padding_pixels:
        res_paths = gen_samples_mp4s(video_frames_samples, class_label, output_dir, padding_pixels=padding_pixel)
        for selected_frame_type in frame_types:
            mmaction_df = mmaction_df.append(pd.DataFrame([(str(path.name), class_labels.index(class_label), padding_pixel, selected_frame_type) for path in res_paths[selected_frame_type]], columns=["path", "label_index", "padding_pixels", "frame_type"]), ignore_index=True)

data_df_path = data_dir/'all_data.txt'
mmaction_df.to_csv(data_df_path, index=False, header=False, sep=' ')


for selected_frame_type in frame_types:
    selected_frame_type_df = mmaction_df[mmaction_df["frame_type"] == selected_frame_type]
    train_df_path = data_dir/f'train_data_{selected_frame_type}.txt'
    test_df_path = data_dir/f'test_data_{selected_frame_type}.txt'

    train_df = selected_frame_type_df.sample(frac=0.8, random_state=0, replace=False)
    test_df = selected_frame_type_df.drop(train_df.index, inplace=False)

    # only keep the path and label_index columns
    train_df = train_df[["path", "label_index"]]
    test_df = test_df[["path", "label_index"]]

    train_df.to_csv(train_df_path, index=False, header=False, sep=' ')
    test_df.to_csv(test_df_path, index=False, header=False, sep=' ')


In [ ]:
# from sklearn.model_selection import train_test_split

# train_df_path = data_dir/'train_data.csv'
# test_df_path = data_dir/'test_data.csv'

# # split train and test from df
# train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
# train_df.to_csv(train_df_path, index=False, header=False, sep=' ')
# test_df.to_csv(test_df_path, index=False, header=False, sep=' ')
